In [ ]:
from config import setup
setup()

In [22]:
import math
from docplex.cp.model import CpoModel
from docplex.cp.model import *

In [5]:
G = [[],[]]    # Notre graphe

X, A = G

X = [[],[],[]] # Ensemble des sommets du graphe

E, T, S = X

E = [] # Ensemble de sommets à évacuer
T = [] # Ensemble de sommets de transfert
S = [] # Ensemble de sommets séurisés

for k in E:
    
    A_k, w_k, p_k, s_k, d_k, h_k = k
    
    i_k = 0  # Identifiant du sommet
    w_k = 0  # Nombre de personnes à évacuer
    h_k = 0  # Taux d'évacuation d'un sommet
    d_k = 0  # Date à laquelle le sommet crame
    p_k = 0  # Durée d'évacuation d'un sommet
    A_k = [] # Liste des sommets pour le chemin d'évacuation
    s_k = 0  # Date début évacuation d'un sommet
    
    for e in A_k:
        
        i_e = 0 # Identifiant de l'arc
        b_e = 0 # Date min de passage

for k in T:
    
    k = 0  # Identifiant d'un sommet de transfert
    
for k in S:
    
    k = 0  # Identifiant d'un sommet sécurisé
    
for e in A:
    
    i_e, in_e, out_e, c_e, b_e = e
    
    i_e   = 0 # Identifiant de l'arc
    c_e   = 0 # Capacité de l'arc en personnes par minute
    in_e  = 0 # Identifiant du sommet entrant
    out_e = 0 # Identifiant du sommet sortant



In [20]:
def read_evac(filename):
    
    with open(filename,"r") as file:
        content = file.readlines()
    
    if len(content) > 0:
        
        nbzones, nbarcs = content[0].split(" ")
        content.remove(content[0])
    
        lineszones = content[0:int(nbzones)]
        linesarcs  = content[int(nbzones):int(nbzones)+int(nbarcs)]
        
        E = []
        
        for line in lineszones:
            personnes, paquets, datemax = line.split(" ")
            E.append([int(personnes),int(paquets),int(datemax)])
            
        A = []
        
        for line in linesarcs:
            
            content = line.split(" ")
            capacite = content[0]
            content = content[2:]
            
            groups = []
            
            for i in range(0,len(content),2):
                numgroupe = content[i]
                datemax = content[i+1]
                
                groups.append([int(numgroupe),int(datemax)])
                
            A.append([int(capacite),groups])
            
        return E,A
        
        
# import os
# os.listdir()

E,A = read_evac("projet/evacsim-master/data/test_10_25_2_10.evac")

print(*E, sep = "\n")
print(*A, sep = "\n")

[1255, 74, 76]
[1578, 70, 105]
[4174, 72, 127]
[1019, 70, 106]
[3395, 72, 99]
[3825, 72, 127]
[2743, 71, 102]
[4384, 132, 107]
[2623, 71, 127]
[1107, 71, 116]
[212, [[0, 16], [1, 11], [3, 10], [4, 39], [5, 15], [6, 14], [7, 9], [8, 14]]]
[132, [[2, 27], [9, 34]]]
[71, [[1, 1], [3, 0], [6, 4]]]
[203, [[0, 31], [1, 26], [2, 36], [3, 25], [4, 54], [5, 30], [6, 29], [7, 24], [8, 29], [9, 43]]]
[70, [[1, 1], [6, 4]]]
[224, [[0, 12], [1, 7], [3, 6], [6, 10], [7, 5], [8, 10]]]
[212, [[0, 8], [1, 3], [3, 2], [6, 6], [7, 1]]]


In [ ]:
def extract_graph(filename):
    
    E,A = read_evac("projet/evacsim-master/data/test_10_25_2_10.evac")
    
    ########################## TRAITEMENT DES SOMMETS ##########################
    
    new_E = {}
    
    for i_k in range(len(E)):
        w_k, h_k, d_k = E[i_k]
        p_k = math.ceil(w_k / h_k) # Durée d'évacuation d'un sommet
        A_k = [] # Liste des sommets pour le chemin d'évacuation
        s_k = None  # Date début évacuation d'un sommet
        
        k = {
            "i_k": i_k,
            "w_k": w_k,
            "h_k": h_k,
            "d_k": d_k,
            "p_k": p_k,
            "A_k": A_k,
            "s_k": s_k,
        }
        
        new_E[i_k] = k
        
    new_T = []
    
    new_S = []
        
    ########################## TRAITEMENT DES ARCS ##########################
    
    new_A = []
    
    for i_e in range (len(A)):
        c_e, groups = A[i_e]
        in_e = None
        out_e = None

        for i_k, b_e in groups:
            
            e = {
                "i_e": i_e,
                "b_e": b_e,
            }
            
            k = new_E.get(i_k)
            A_k = k.get("A_k")
            
            if len(A_k) == 0:
                A_k.append(e)
            else:
                # TODO Parcourir A_k et insérer à la bonne date



    
    ########################## TRAITEMENT DU GRAPHE ##########################
    
    X = {
        "E": new_E,
        "T": new_T,
        "S": new_S,
    }
    
    G = {
        "X": X,
        "A": new_A,
    }
    
    return G

G = extract_graph("projet/evacsim-master/data/test_10_25_2_10.evac")

import json
print(json.dumps(G, indent=1))


{
 "X": {
  "T": [],
  "E": {
   "0": {
    "s_k": null,
    "p_k": 17,
    "w_k": 1255,
    "A_k": [],
    "d_k": 76,
    "h_k": 74,
    "i_k": 0
   },
   "1": {
    "s_k": null,
    "p_k": 23,
    "w_k": 1578,
    "A_k": [],
    "d_k": 105,
    "h_k": 70,
    "i_k": 1
   },
   "2": {
    "s_k": null,
    "p_k": 58,
    "w_k": 4174,
    "A_k": [],
    "d_k": 127,
    "h_k": 72,
    "i_k": 2
   },
   "3": {
    "s_k": null,
    "p_k": 15,
    "w_k": 1019,
    "A_k": [],
    "d_k": 106,
    "h_k": 70,
    "i_k": 3
   },
   "4": {
    "s_k": null,
    "p_k": 48,
    "w_k": 3395,
    "A_k": [],
    "d_k": 99,
    "h_k": 72,
    "i_k": 4
   },
   "5": {
    "s_k": null,
    "p_k": 54,
    "w_k": 3825,
    "A_k": [],
    "d_k": 127,
    "h_k": 72,
    "i_k": 5
   },
   "6": {
    "s_k": null,
    "p_k": 39,
    "w_k": 2743,
    "A_k": [],
    "d_k": 102,
    "h_k": 71,
    "i_k": 6
   },
   "7": {
    "s_k": null,
    "p_k": 34,
    "w_k": 4384,
    "A_k": [],
    "d_k": 107,
    "h_k": 132

In [ ]:
def geosafe_model(G, ...):

    mdl = CpoModel(name='geosafe')
    
    s = mdl.integer_var_list(G.E.shape[0], "Date début évacuation du sommet ")
    
    h = mdl.integer_var_list(G.E.shape[0], "Taux d'évacuation du sommet ")
    

5
